# Technical Details

As with all packages, there are numerous technical details that are abstracted away from the user. Now in order to ensure a clean interface, this abstraction is entirely necessary. However, it can sometimes be confusing when navigating a package's source code to pin down what's going on when there's so many _under the hood_ operations taking place. In this notebook I'll aim to shed some light on all of tricks that we do in GPJax in order to help elucidate the code to anyone wishing to extend GPJax for their own uses.


## Parameter Transformations

### Motivations

Many parameters in a Gaussian process are what we call a _constrained parameter_. By this, we mean that the parameters value is only defined on a subset of $\mathbb{R}$. One example of this is the lengthscale parameter in any of the stationary kernels. It would not make sense to have a negative lengthscale, and as such the parameter's value is constrained to exist only on the positive real line. 

Whilst mathematically correct, constrained parameters can become a pain when optimising as many optimisers are designed to operate on an unconstrained space. Further, it can often be computationally inefficient to restrict the search space of an optimiser. For these reasons, we instead transform the constrained parameter to exist in an unconstrained space. Optimisation is then done on this unconstrained parameter before we transform it back when we need to evaluate its value. 

Only bijective transformations are valid as we cannot afford to lose our original parameter value when transforming. As such, we have to be careful about which transformations we use. Some common choices include the log-exponential bijection and the softplus transform. We, by default, opt for the softplus transformation in GPJax as it less prone to overflowing in comparison to log-exp transformations.


### Implementation

When it comes to implementations, we attach the transformation directly to the `Parameter` class. It is an optional argument that one can specify when instantiating their parameter. To see this, simply consider the following example

In [6]:
from gpjax.parameters import Parameter
from gpjax.transforms import Softplus
import jax.numpy as jnp

x = Parameter(jnp.array(1.0), transform=Softplus())

Now we know that the softplus transformation operation on an input $x \in \mathbb{R}_{>0}$ can be written as 
$$\alpha(x) = \log(\exp(x)-1)$$
where $\alpha(x) \in \mathbb{R}$. In this instance, it can be seen that $\alpha(1)=0.54$. Now this unconstrained value is stored within the parameter's `value` property

In [7]:
print(x.value)

0.541324854612918


whilst the original constrained value can be computed by accesing the parameter's `untransform` property

In [8]:
print(x.untransform)

1.0


### Custom transformation

Should you wish to define your own custom transformation, then this can very easily be done by simply extending the `Transform` class within `gpjax.transforms` and defining a forward transformation and a backward transformation.

In [9]:
class Transform:
    def __init__(self, name="Transformation"):
        self.name = name

    @staticmethod
    def forward(x: jnp.ndarray) -> jnp.ndarray:
        raise NotImplementedError

    @staticmethod
    def backward(x: jnp.ndarray) -> jnp.ndarray:
        raise NotImplementedError

The `forward` method is the transformation that maps from a constrained space to an unconstrained space, whilst the `backward` method is the transformation that reverses this. A nice example of this can be seen for the earlier used softplus transformation

In [10]:
from jax.nn import softplus

class Softplus(Transform):
    def __init__(self):
        super().__init__(name='Softplus')

    @staticmethod
    def forward(x: jnp.ndarray) -> jnp.ndarray:
        return jnp.log(jnp.exp(x) - 1.)

    @staticmethod
    def backward(x: jnp.ndarray) -> jnp.ndarray:
        return softplus(x)

## Prior distributions

### Motivations

Often when we use Gaussian processes, we do so as they facilitate easily incorporation of prior information into the model. Implicitly, by the very use of a Gaussian process we are incorporating our prior inforation around the functional behaviour of the latent function that we are seeking to recover. However, we can take this one step further by placing priors on the hyperparameters of the Gaussian process. Going into the details of which priors are recommended and how to go about selecting them goes beyond the scope of this article, but it's suffice to say that doing so can greatly enhance the utility of a Gaussian process. 

At least in my own experience, when priors are placed on the hyperparameters of a Gaussian process they are specified with respect to the constrained parameter value. As an example of this, consider the lengthscale parameter $\ell \in \mathbb{R}_{>0}$. When specifying a prior distribution $p_{0}(\ell)$, I would typically select a distribution that has support on the positive real line, such as the Gamma distribution. An opposing approach would be to transform the parameter so that it is defined on the entire real line (as discussed in §1) and then specify a prior distribution such as a Gaussian that has an unconstrained support. Deciding which of these two approaches to adopt in GPJax is somewhat a moot point to me, so I've opted for priors to be defined on the constrained parameter. That being said, I'd be more than open to altering this opinion is people felt strongly that priors should be defined on the unconstrained parameter value.

### Implementation

Regarding the implementational details of enabling prior specification, this is hopefully a more lucid concept upon code inspection. As with the earlier discussed parameter transformations, the notion of a prior distribution is acknolwedged in the definition of a parameter. To exactly specify a prior distribution, one should simply call in the relevant distribution from TensorFlow probability's distributions module. For an example of this, consider the parameter `x` that was earlier defined.

In [22]:
from tensorflow_probability.substrates.jax import distributions as tfd

x.prior = tfd.Gamma(concentration = 3., rate = 2.)

If we momentarily pause to consider the state of this parameter now, then we have a constrained parameter value with corresponding prior distribution. When it comes to deriving our posterior distribution, then we know that it is proportional to the product of the likelihood and the prior density function. As addition is less prone to numerical overflow than multiplication, we take the log of this produce. The log of a product is just a sum of logs, meaning that our log-posterior is then proportional to the sum of our log-likelihood and the log-prior density. Therefore, to connect the value of our parameter and its respective prior distribution, the only implementational point left to cover is how to evaluate the parameters log-prior density. This can be done through the following `@property`

In [24]:
print(x.log_density)

-0.613706111907959


Naturally, should one wish to evaluate the prior density of the parameter, then the exponent can be taken

In [26]:
print(jnp.exp(x.log_density))

0.5413408768770793


## Cholesky decomposition

